<img style="float: left;;" src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20II/Figures/alinco.png?raw=1' /></a>

# Modulo I: Naive Bayes para Análisis de Sentimientos


Aplicaremos Naive Bayes para el análisis de sentimientos de tweets, lo que haremos en este notebook será:

* Entrenar un modelo de Naive Bayes para el análisis de sentimientos
* Probaremos el modelo creado
* Calcularemos las proporciones de palabras positivas a palabras negativas
* Veremos una función de análisis de errores
* Predeciremos un tweet


In [1]:
from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

Si aún no se han descargado:
```
nltk.download('stopwords')
nltk.download('twitter_samples')
```

In [2]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [3]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [4]:
# get the sets of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# avoid assumptions about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

# Preprocesamiento de los datos

Para cualquier proyecto de aprendizaje automático, una vez que se haya recopilado los datos, el primer paso es el preprocesamiento.

- **Eliminar el ruido**: eliminar las palabras que no dicen mucho sobre el contenido. Estos incluyen todas las palabras comunes como 'I, you, are, is, etc...' que no nos darían suficiente información sobre el sentimiento.
- También eliminaremos los tickers del mercado de valores, los símbolos de retuits, los hipervínculos y los hashtags porque no pueden brindarle mucha información sobre el sentimiento.
- También hay que eliminar toda la puntuación de un tweet. La razón para hacer esto es porque queremos tratar las palabras con o sin puntuación como la misma palabra, en lugar de tratar las palabras "happy", "happy?", "happy!", "happy," and "happy." como palabras diferentes.
- Por último, desea utilizaremos la técnica de stemming para realizar un seguimiento de una sola variación de cada palabra. En otras palabras, trataremos "motivation", "motivated", y "motivate" de manera similar agrupándolos dentro de la misma raíz de "motiv-".

Recuerden que ya tenemos una función para esto (`process_tweet()`)

In [5]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# Utilizar el metodo prep
from utilss import Utilities as prep
obj_prep = prep()
print(obj_prep.process_tweet(custom_tweet))

This is the Utilities Constructor
['hello', 'great', 'day', ':)', 'good', 'morn']


## Funciones de ayuda

Para entrenar el modelo, necesitaremos construir un diccionario donde las claves sean una tupla (word, label) y los valores sean la frecuencia correspondiente. Tenga en cuenta que las etiquetas que usaremos aquí son 1 para positivo y 0 para negativo.

Implementaremos la función `lookup()` que toma del diccionario `freqs` una palabra y una etiqueta (1 or 0) y retorna el número de veces que la palabra y esa etiqueta aparecen en la collección de tweets.

Por ejemplo: Dado una lista de sentencias en los tweets: `["i am rather excited", "you are rather happy"]` y la etiqueta 1, la función regresara el diccionario que contiene lo siguiente:

{
    ("rather", 1): 2
    ("happi", 1) : 1
    ("excit", 1) : 1
}

- Observe cómo para cada palabra en la cadena dada, se asigna la misma etiqueta 1 a cada palabra.
- Observe cómo las palabras "i" y "am" no se guardan, ya que fue eliminada por process_tweet porque es una stopword.
- Observe cómo la palabra "rather" aparece dos veces en la lista de tweets, por lo que su valor de recuento es 2.

Crearemos una función `count_tweets()` que toma una lista de tweets como entrada, las limpia, y regresa un diccionario.

- La clave en el diccionario es una tupla que contiene la palabra despues del stemming y su etiqueta de clase, Ej. ("happi", 1).
- El valor del key, será la cantidad de veces que esta palabra aparece en la colección de tweets dada (un número entero).


In [6]:
def count_tweets(result, tweets, ys):
    '''
    Input:
    result: un diccionaio que nos va servir para mapear las frecuencias de las palabras
    tweets: la lista de tweets
    ys: el sentimiento (positivo->1, negativo->0)
    output:
    result
    '''

    for y, tweet in zip(ys, tweets):
      for word in obj_prep.process_tweet(tweet):
        #definir el key, el cual es la palabra y la etiqueta en una tupla
        pair = (word, y)

        # si el key existe en el diccionario, hay que incrementar el conteo
        if pair in result:
          result[pair] +=1
        else:
          result[pair] = 1

    return result

In [7]:
results = {}
tweets = ['I am happy', "I am tricked", "I am sad", 'I am tired']
ys = [1, 0, 0, 0]

In [8]:
# Testing your function
r = count_tweets(results, tweets, ys)
r


{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 1}

## Entrenar el modelo usando Naive Bayes

Naive bayes es un algoritmo que se puede usar para aplicaciones como el análisis de sentimientos.


#### ¿Cómo entrenamos un clasificador con Naive Bayes ?

- La primera parte del entrenamiento de un clasificador Naive Bayes es identificar el número de clases que tiene.
- Crearemos una probabilidad para cada clase.


$P(D_{pos})$ denota la probabilidad que el documento sea positivo.
$P(D_{neg})$ denota la probabilidad que el documento sea negativo.

Utilizaremos las fórmulas de la siguiente manera y almacenaremos los valores en un diccionario:

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{neg}) = \frac{D_{neg}}{D}\tag{2}$$

Donde $ D $ es el número total de documentos, o tweets en este caso, $ D_ {pos} $ es el número total de tweets positivos y $ D_ {neg} $ es el número total de tweets negativos.

#### Prior y Logprior

La probabilidad previa (prior) representa la probabilidad subyacente de la población (tweets) de que un tweet sea positivo frente a uno negativo. En otras palabras, si no tuviéramos información específica y seleccionáramos a ciegas un tweet del conjunto de la población, ¿cuál es la probabilidad de que sea positivo frente a que sea negativo? Ese es el "previo".

El "prior" es el ratio de las probabilidades $\frac{P(D_{pos})}{P(D_{neg})}$. Podemos tomar el logaritmo del "prior" para cambiar su escala, y lo llamaremos logprior.


$$\text{logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$$.

Note que $log(\frac{A}{B})$ es igual a $log(A) - log(B)$.  Por lo tanto logprior puede calcularse como la diferencia de dos logaritmos (propiedad de logaritmos).


$$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\tag{3}$$

#### Probabilidad positiva y negativa de una palabra
Para calcular la probabilidad positiva y la probabilidad negativa de una palabra específica en el vocabulario, usaremos las siguientes notaciones:

- $freq_{pos}$ y $freq_{neg}$ son las frecuencias de esa palabra específica en la clase positiva o negativa. En otras palabras, la frecuencia positiva de una palabra es el número de veces que se cuenta la palabra con la etiqueta 1.
- $ N_ {pos} $ y $ N_ {neg} $ son el número total de palabras positivas y negativas para todos los documentos (para todos los tweets), respectivamente.

- $V$ es el número de palabras únicas en todo el conjunto de documentos, para todas las clases, ya sean positivas o negativas.

Los usaremos para calcular la probabilidad positiva y negativa de una palabra específica usando la fórmula:

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

Note que añadimos "+1" en el numerador para el smoothing aditivo.  Este [artículo](https://en.wikipedia.org/wiki/Additive_smoothing) explica detalladamente sobre el smoothing aditivo.

#### Log likelihood
Para calcular la loglikelihood (probabilidad mínima) de esa misma palabra, podemos implementar las siguientes ecuaciones:

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Creación del diccionario de  frecuencias (`freqs` )
- Dada la función `count_tweets ()`, podemos calcular un diccionario llamado `freqs` que contiene todas las frecuencias.
- En este diccionario `freqs`, la clave es la tupla (word,label)
- El valor es el número de veces que ha aparecido.



In [9]:
#obtener el diccionario
freqs = count_tweets({}, train_x, train_y)


In [10]:
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

### Pipeline para la creación de la función  `train_naive_bayes`
Dado un diccionario de frecuencias, `train_x` (una lista de tweets) y un` train_y` (una lista de etiquetas para cada tweet), implementaremos un clasificador Naive Bayes.

##### Calcular $V$
- Podemos calcular el número de palabras únicas que aparecen en el diccionario `freqs` para obtener $ V $ (usando la función` set`).

##### Calcular $freq_{pos}$ y $freq_{neg}$
- Usando el diccionario `freqs`, se puede calcular la frecuencia positiva y negativa de cada palabra $ freq_ {pos} $ y $ freq_ {neg} $.

##### Calcular $N_{pos}$ y $N_{neg}$
- Usando el diccionario `freqs`, también puede calcular el número total de palabras positivas y el número total de palabras negativas $ N_ {pos} $ y $ N_ {neg} $.

##### Calcular $D$, $D_{pos}$, $D_{neg}$
- Usando la lista de etiquetas de entrada `train_y`, calcularemos el número de documentos (tweets) $ D $, así como el número de documentos positivos (tweets) $ D_ {pos} $ y el número de documentos negativos (tweets) $ D_ { neg} $.
- Calcularemos la probabilidad de que un documento (tweet) sea positivo $ P (D_ {pos}) $, y la probabilidad de que un documento (tweet) sea negativo $ P (D_ {neg}) $

##### Calcularemos el logprior
-  $log(D_{pos}) - log(D_{neg})$

##### Calculate log likelihood
- Finalmente, iteramos sobre cada palabra en el vocabulario, usaremos la función `lookup` para obtener las frecuencias positivas, $ freq_ {pos} $, y las frecuencias negativas, $ freq_ {neg} $, para esa palabra específica.
- Calcularemos la probabilidad positiva de cada palabra $ P (W_ {pos}) $, la probabilidad negativa de cada palabra $ P (W_ {neg}) $ usando las ecuaciones 4 y 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

**Note:** Usaremos un diccionario para almacenar los loglikelihoods de cada palabra. La clave será la palabra, el valor será la probabilidad logarítmica de esa palabra).

- se puede calcular la loglikelihood: $log \left( \frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$.

In [11]:
[pair[0] for pair in freqs.keys()]


['followfriday',
 'top',
 'engag',
 'member',
 'commun',
 'week',
 ':)',
 'hey',
 'jame',
 'odd',
 ':/',
 'pleas',
 'call',
 'contact',
 'centr',
 '02392441234',
 'abl',
 'assist',
 'mani',
 'thank',
 'listen',
 'last',
 'night',
 'bleed',
 'amaz',
 'track',
 'scotland',
 'congrat',
 'yeaaah',
 'yipppi',
 'accnt',
 'verifi',
 'rqst',
 'succeed',
 'got',
 'blue',
 'tick',
 'mark',
 'fb',
 'profil',
 '15',
 'day',
 'one',
 'irresist',
 'flipkartfashionfriday',
 'like',
 'keep',
 'love',
 'custom',
 'wait',
 'long',
 'hope',
 'enjoy',
 'happi',
 'friday',
 'lwwf',
 'second',
 'thought',
 '’',
 'enough',
 'time',
 'dd',
 'new',
 'short',
 'enter',
 'system',
 'sheep',
 'must',
 'buy',
 'jgh',
 'go',
 'bayan',
 ':d',
 'bye',
 'act',
 'mischiev',
 'etl',
 'layer',
 'in-hous',
 'wareh',
 'app',
 'katamari',
 'well',
 '…',
 'name',
 'impli',
 ':p',
 'influenc',
 'big',
 '...',
 'juici',
 'selfi',
 'follow',
 'perfect',
 'alreadi',
 'know',
 "what'",
 'great',
 'opportun',
 'junior',
 'triathle

In [12]:
vocab = set([pair[0] for pair in freqs.keys()])
vocab

{'siguro',
 'minha',
 'dey',
 'impair',
 'follow',
 '😉',
 'netizen',
 'nathanielhinanakit',
 'except',
 "year'",
 'thunderstorm',
 'hahah',
 'uuughhh',
 'movement',
 'cauliflow',
 'write',
 'tologooo',
 'elf',
 'thistlelov',
 'song',
 'gw',
 'planner',
 'weloveyounamjoon',
 'owww',
 'veggi',
 'clinton',
 'boi',
 'scroll',
 'vandag',
 'aspetti',
 'olli',
 'carolin',
 'confid',
 '🎀',
 'ign',
 'previou',
 'hash',
 '.\n.\n.',
 'gryph',
 'romanc',
 'honest',
 'superdri',
 '318',
 'holborn',
 'non-mapbox',
 'bastard',
 'gay',
 '💜',
 'rihanna',
 'creak',
 'addmeonbbm',
 'mwa',
 'pleass',
 'bear',
 'kita',
 'sponsor',
 'ng',
 'seven',
 'infront',
 'vandr',
 'gobigorgohom',
 'oitnb',
 'carriag',
 'ho',
 'kca',
 'search',
 'yhm',
 'shine',
 'centr',
 'one',
 'franc',
 'kristin',
 'inc',
 '2ish',
 'anonym',
 'day',
 'op',
 'pinkeu',
 'airforc',
 'froze',
 '3:02',
 'quickest',
 'besok',
 'akana',
 '26week',
 'campaign',
 'hotti',
 'hotel',
 'famou',
 'ink',
 'smashingbook',
 'selen',
 'stretch',
 

In [13]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    inputs:
    freqs: un diccionario con (w,l) y con el key con la frecuencia que aparece la palabra en ese tweet
    train_x: lista de tweets para el entrenamiento
    train_y: lista de las etiquetas de los tweets(0,1)

    outputs:
    logprior: lo prior (log(Dpos) - log(Dneg))
    loglikelihood: es el loglikelihood de la ecuación de naive bayes log(P(Wpos)/P(Wneg))
    '''

    loglikelihood = {}
    logprior = 0

    # 1.- calcular V, el número de valores únicos en el vocabulario
    vocab = set([pair[0] for pair in freqs.keys()])
    V=len(vocab)

    #2.- Calcular N_pos, n_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
      #si la etiqueta es positiva mayor a 0
      if pair[1] > 0:
        N_pos += freqs[pair] #incrementar el número de palabras positivas en el conteo de (w,l)
      else:  #La etiqueta es negativa
        N_neg += freqs[pair] #incrementar el número de palabras negativas por el conteo de (w,l)

    #Calcular el valor de D, el número de tweets (documentos)
    D = len(train_y)

    # Calcular D_pos, el número de documentos positivos
    D_pos = len(list(filter(lambda x: x>0, train_y)))

    # Calcular D_neg, el número de documentos negativos
    D_neg = D - D_pos

    # Calcular el logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    #Calcular loglikelihood
    for word in vocab:
      #obtener las frecuencias positivas y negativas de cada palabra
      freq_pos = lookup(freqs, word, 1)
      freq_neg = lookup(freqs, word, 0)

      # Calcular la probabilidad de que cada palabra sea un sentimiento positivo, y lo mismo para el sentimiento negativo
      # Suavizado
      p_w_pos = (freq_pos + 1) / (N_pos + V)
      p_w_neg = (freq_neg + 1) / (N_neg + V)

      #Calculamos el likelihood de cada palabra
      loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    return logprior, loglikelihood


In [14]:
# probar train
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)


In [17]:
print(logprior)

0.0


In [18]:
len(loglikelihood)

9085

In [19]:
loglikelihood

{'siguro': 0.6982557485026679,
 'minha': -0.6880386126172228,
 'dey': -1.0935037207253873,
 'impair': 0.6982557485026679,
 'follow': 0.2680980278041318,
 '😉': 0.41057367605088696,
 'netizen': 0.6982557485026679,
 'nathanielhinanakit': -0.6880386126172228,
 'except': -0.6880386126172228,
 "year'": 1.1037208566108323,
 'thunderstorm': 0.6982557485026679,
 'hahah': -1.7866509012853324,
 'uuughhh': -0.6880386126172228,
 'movement': -0.6880386126172228,
 'cauliflow': 0.6982557485026679,
 'write': 0.2927906403945036,
 'tologooo': 0.6982557485026679,
 'elf': 0.00510856794272258,
 'thistlelov': 0.6982557485026679,
 'song': -0.4197746260225436,
 'gw': -0.6880386126172228,
 'planner': -0.6880386126172228,
 'weloveyounamjoon': 0.00510856794272258,
 'owww': -0.4003565401654419,
 'veggi': 0.6982557485026679,
 'clinton': 0.6982557485026679,
 'boi': 0.6982557485026679,
 'scroll': -0.6880386126172228,
 'vandag': -0.6880386126172228,
 'aspetti': 0.6982557485026679,
 'olli': 1.1037208566108323,
 'caroli

# Probando el modelo de Naive Bayes

Ahora que tenemos `logprior` y` loglikelihood`, podemos probar el modelo de Naive Bayes para predecir algunos tweets.

#### Implementación de  `naive_bayes_predict`

Implementaremos la función `naive_bayes_predict` para hacer predicciones de tweets.

* La función toma como entrada el `tweet`,` logprior`, y `loglikelihood`.
* Devuelve la probabilidad de que el tweet pertenezca a la clase positiva o negativa.
* Para cada tweet, se sumarán las probabilidades de cada palabra en el tweet.
* También se agregará el logprior de esta suma para obtener la predicción de sentimiento del tweet

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Nota

Tenga en cuenta que calculamos el "prior" a partir de los datos de entrenamiento y que los datos de entrenamiento se dividen uniformemente entre etiquetas positivas y negativas (4000 tweets positivos y 4000 negativos). Esto significa que el ratio de positivo a negativo 1, y el logprior es 0.

El valor de 0.0 significa que cuando agregamos el logprior al likelihood, simplemente estamos agregando un valor de cero a la probabilidad logarítmica. Sin embargo, recuerde incluir el logprior, porque siempre que los datos no estén perfectamente equilibrados, el logprior será un valor distinto de cero.


In [20]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    input:
    tweet: una cadena de caracteres
    logprior. un número (0 si el dataset está balanceado)
    loglikelihood: es un diccionario de palabras que mapean a números
    output:
    p: la suma de todos los loglikelihoods de cada palabra en el tweet
    '''

    #preprocesamiento del tweet
    word_l = process_tweet(tweet)

    p = 0
    #Añadir a p, el logprior
    p += logprior

    for word in word_l:
      #Checar si la palabra existe en el loglikelihood (el diccionario)
      if word in loglikelihood:
        #añadimos el valor a la probabilidad de p
        p +=loglikelihood[word]
    return p


In [24]:
#predecir
my_tweet = "she Smiled a lot"
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
p

3.0288432293052887

In [25]:
my_tweet = "yor are so bad"
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
p

-1.2941744161875384

In [26]:
x_tweet = 'A thousand planets. One mission to save them all. Watch the new #Valerian trailer and like this Tweet to explore more. In theaters July 21'
p = naive_bayes_predict(x_tweet, logprior, loglikelihood)
p

2.814785778290656

#### Implementando  test_naive_bayes

* Implementaremos `test_naive_bayes` la función para ver el accuracy de las predicciones.
* La función toma de `test_x`, `test_y`,el log_prior, y el loglikelihood
* Regresa el ccuracy del modelo
* Primeramente, usaremos la función `naive_bayes_predict` para hacer las predicciones para cada tweet en text_x.

In [27]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    '''
    Input:
    test_x. lista de tweets a entrenar
    test_y: las etiquetas de positivos y negativos
    logprior: el logprior (log(Dpos) - log(Dneg))
    loglikelihood. diccionario con las palabras y el loglikelihood

    '''
    accuracy=0

    y_hats = []
    for tweet in test_x:
      if naive_bayes_predict(tweet, logprior, loglikelihood) >0:
        #el tweet pertece a un sentimiento positivo
        y_hats.append(1)
      else:
        #el tweet pertenece a un sentimiento negativo o neutro
        y_hats.append(0)

    error = np.mean(np.absolute(y_hats - test_y))

    accuracy = 1- error
    return accuracy


In [29]:
print('Accuracy: ', test_naive_bayes(test_x, test_y, logprior, loglikelihood))

Accuracy:  0.994


In [30]:
tweets=['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']
for tweet in tweets:
  p = naive_bayes_predict(tweet, logprior, loglikelihood)
  print(f'{tweet} ----> {p:2f}')


I am happy ----> 2.148266
I am bad ----> -1.294174
this movie should have been great. ----> 2.142903
great ----> 2.137795
great great ----> 4.275589
great great great ----> 6.413384
great great great great ----> 8.551178


### Predecir un tweet propio

In [ ]:
# Feel free to check the sentiment of your own tweet below


# Filtrado de palabras por ratios positivos y negativos

- Algunas palabras tienen un conteo de etiquetas más positivos que otras y pueden considerarse "más positivas". Asimismo, algunas palabras pueden considerarse más negativas que otras.

- Una forma de definir el nivel de positividad o negatividad, sin calcular la probabilidad logarítmica, es comparar la frecuencia positiva con la negativa de la palabra.

    - Tenga en cuenta que también podemos utilizar los cálculos de probabilidad logarítmica para comparar la positividad o negatividad relativa de las palabras.
    
- Podemos calcular el ratio de las frecuencias positivas y negativas de una palabra.
- Una vez que podamos calcular estos ratios, también podemos filtrar un subconjunto de palabras que tengan una proporción mínima de positividad / negatividad o superior.
- De manera similar, también podemos filtrar un subconjunto de palabras que tienen una proporción máxima de positividad / negatividad o menor (palabras que son menos negativas, o incluso más negativas que un umbral dado).

#### Implementación `get_ratio()`

- Dado el diccionario de palabras `freqs` y una palabra en particular, usaremos` lookup (freqs, word, 1) `para obtener el recuento positivo de la palabra.
- De manera similar, usaremos la función `lookup ()` para obtener el recuento negativo de esa palabra.
- Calcularemos la proporción de positivo dividido por conteos negativos.

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

Where pos_words and neg_words correspond to the frequency of the words in their respective classes.
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

In [31]:

def get_ratio(freqs, word):
    '''
    Input:
    freqs : el diccionario que contienne las palabras
    word: la palabra a buscar
    Output:
    pos_neg_ratio: un diccionario con los keys 'positivos', 'negativos' y 'ratio
    ej: {'positivo':0, 'negativo':0, 'ratio': 0}
    '''

    pos_neg_ratio = {'positivo': 0, 'negativo': 0, 'ratio': 0.0}

    #usar el método lookup() para encontrar el conteo de las palabras que son negativas
    pos_neg_ratio['negativo'] = lookup(freqs,word, 0)

    #usar el método lookup() para encontrar el conteo de las palabras que son positivas
    pos_neg_ratio['positivo'] = lookup(freqs,word, 1)

    pos_neg_ratio['ratio'] = (pos_neg_ratio['positivo'] + 1)/(pos_neg_ratio['negativo'] + 1)

    return pos_neg_ratio


In [32]:
get_ratio(freqs, 'happi')

{'positivo': 161, 'negativo': 18, 'ratio': 8.526315789473685}

#### Implementación `get_words_by_threshold(freqs,label,threshold)`

* Si establecemos label en 1, buscaremos todas las palabras cuyo umbral de positivo / negativo sea mayoy o igual al umbral.
* Si establecemos la etiqueta en 0, buscaremos todas las palabras cuyo umbral de positivo / negativo sea menor o igual al umbral.
* Utilizaremos la función `get_ratio ()` para obtener un diccionario que contenga el recuento positivo, el recuento negativo y la proporción de recuentos positivos y negativos.
* Agregaremos un diccionario a una lista, donde la clave es la palabra y el diccionario es el diccionario `pos_neg_ratio` que es devuelto por la función` get_ratio () `.

Un ejemplo de este diccionario sería:
```
{'happi':
    {'positive': 10, 'negative': 20, 'ratio': 0.5}
}
```

In [33]:

def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
    freqs. diccionario de palabras con sus frecuencias
    label: etiqueta para el sentimiento (1-> positivo, 0->negativo)
    threshold: ratio que se va usar para filtrar las palabras del diccionario
    '''

    word_list = {}
    for key in freqs.keys():
      word, _ = key
      #los ratios de cada palabra (pos/neg)
      pos_neg_ratio = get_ratio(freqs, word)

      #si la etiqueta es 1 y el ratio es mayor al threshold entonces se agrefa al diccionario word_list
      if label==1 and pos_neg_ratio['ratio']>=threshold:
        word_list[word] = pos_neg_ratio
      elif label==0 and pos_neg_ratio['ratio']<threshold:
        word_list[word] =pos_neg_ratio

    return word_list


In [34]:
# Testear la función con etiqueta 0
get_words_by_threshold(freqs, label=0, threshold = 0.05)

{':(': {'positivo': 1, 'negativo': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positivo': 0, 'negativo': 378, 'ratio': 0.002638522427440633},
 '26': {'positivo': 0, 'negativo': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positivo': 0, 'negativo': 43, 'ratio': 0.022727272727272728},
 '♛': {'positivo': 0, 'negativo': 210, 'ratio': 0.004739336492890996},
 '》': {'positivo': 0, 'negativo': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positivo': 0, 'negativo': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positivo': 0, 'negativo': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positivo': 0, 'negativo': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positivo': 0, 'negativo': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positivo': 0, 'negativo': 35, 'ratio': 0.027777777777777776}}

In [36]:
# Testear la función con etiqueta 1
get_words_by_threshold(freqs, label=1, threshold = 0.05)

{'followfriday': {'positivo': 23, 'negativo': 0, 'ratio': 24.0},
 'top': {'positivo': 30, 'negativo': 5, 'ratio': 5.166666666666667},
 'engag': {'positivo': 7, 'negativo': 0, 'ratio': 8.0},
 'member': {'positivo': 14, 'negativo': 6, 'ratio': 2.142857142857143},
 'commun': {'positivo': 27, 'negativo': 1, 'ratio': 14.0},
 'week': {'positivo': 72, 'negativo': 47, 'ratio': 1.5208333333333333},
 ':)': {'positivo': 2847, 'negativo': 2, 'ratio': 949.3333333333334},
 'hey': {'positivo': 60, 'negativo': 20, 'ratio': 2.9047619047619047},
 'jame': {'positivo': 7, 'negativo': 3, 'ratio': 2.0},
 'odd': {'positivo': 2, 'negativo': 2, 'ratio': 1.0},
 ':/': {'positivo': 5, 'negativo': 8, 'ratio': 0.6666666666666666},
 'pleas': {'positivo': 80, 'negativo': 243, 'ratio': 0.3319672131147541},
 'call': {'positivo': 27, 'negativo': 22, 'ratio': 1.2173913043478262},
 'contact': {'positivo': 4, 'negativo': 7, 'ratio': 0.625},
 'centr': {'positivo': 1, 'negativo': 2, 'ratio': 0.6666666666666666},
 '0239244123

**Observe la diferencia entre los ratios positivos y negativos. Emojis como :( y palabras como 'me' tienden a tener una connotación negativa. Otras palabras como 'glad', 'comunity' y 'arrives' tienden a encontrarse en los tw-eets positivos.**


# Análisis de Error

En esta parte, veremos algunos tweets que el modelo no clasificó correctamente. ¿Por qué crees que ocurrieron las clasificaciones erróneas? ¿Hubo alguna suposición hecha por el modelo de Naive Bayes?

# Predice con tu propio tweet


In [ ]:
# Testear con un propio tweet